# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from dateutil.parser import parse
from tkinter import*
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox, filedialog
import pickle
import time
import os
import sys
import pyodbc
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import pytz
mydata = []

# # Database connection to MSSQL Server

In [2]:
#================= Create Database Connection ======================#
engine = pyodbc.connect("DRIVER={SQL Server};"
                      "Server=192.168.37.128;"
                      "UID=sa;"
                      "PWD=Abc12345;"
                      "Database=PREDICTIVE;"
                      "Trusted_Connection=NO;")
cursor =engine.cursor()

pd.options.display.float_format = '{:,.2f}'.format

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server does not exist or access denied. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')

# Define function

In [ ]:
def getrow(event):
    rowid = trv.identify_row(event.y)
    item = trv.item(trv.focus())
    
    varNitro.set(item['values'][1])
    varPhos.set(item['values'][2])
    varPot.set(item['values'][3])
    varTemp.set(item['values'][4])
    varHum.set(item['values'][5])
    varPH.set(item['values'][6])
    varRain.set(item['values'][7])
    varCrop.set(item['values'][8]) 
    

In [ ]:
def update(rows):
    global mydata
    mydata=rows
    trv.delete(*trv.get_children())
    for i in rows:
        trv.insert('', 'end', values=i) 

In [ ]:
def simulation():
    dataNitro = varNitro.get()
    dataPhos = varPhos.get()
    dataPot = varPot.get()
    dataTemp = varTemp.get()
    dataHum = varHum.get()
    dataPH = varPH.get()
    dataRain = varRain.get()
    dataCrop = varCrop.get()
    
    if messagebox.askyesno("Calculate Regression", "Start regression Model?"):
        
        with open('RandomForest.pkl', 'rb') as RF:
            model=pickle.load(RF)
            simulation = model.predict([[dataNitro, dataPhos, dataPot, dataTemp,dataHum,dataPH,dataRain]])
            #simulation = np.exp(simulation)
            varCrop.set(simulation)
            
    else:
        return True

In [ ]:
def clear():
    query = "SELECT Nitrogen,Phosphorus,Pottasium,temperature,humidity,ph,rainfall,Crop FROM Crop_recommendation"
    cursor.execute(query)
    rows=cursor.fetchall()
    update(rows)

In [ ]:
    
def exit_program():
    if messagebox.askyesno("Exit Program", "Are you sure you want to exit? All progress will be saved"):
        root.destroy()
    else:
        return True

In [ ]:
def exportcsv():
    if len(mydata)<1:
        messagebox.showerror("Warning", "No data available to export")
        return False
    fln = filedialog.asksaveasfilename(initialdir=os.getcwd(), title="Export CSV", filetypes=(("CSV File","*.csv"), ("All Files", "*.*")))
    with open(fln, mode='w') as myfile:
        exp_writer = csv.writer(myfile, delimiter=',')
        for i in mydata:
            exp_writer.writerow(i)
    messagebox.showinfo("Data Exported", "Your data has been exported to "+os.path.basename(fln)+" successfully.")

# Create Windows Main Frame

In [ ]:
root=Tk()
root.title("AGRICULTURE CROP RECOMMENDATION SYSTEM")
root.geometry("900x555")
root.configure(bg='white')
root.resizable(True, True)

# Declare Variable String 

In [ ]:
varNitro =DoubleVar()
varPhos = DoubleVar()
varPot =  DoubleVar()
varTemp =  DoubleVar()
varHum = DoubleVar()
varPH =  DoubleVar()
varRain = DoubleVar()
varCrop = StringVar()


varNitro.set("0")
varPhos.set("0")
varPot.set("0")
varTemp.set("0")
varHum.set("0")
varPH.set("0")
varRain.set("0")
varCrop.set(" ")



# Create Wrapper Segment

In [ ]:
wrapper1=LabelFrame(root, text="Existing Data Preview", cursor='dot')
wrapper1.pack(fill="both", expand="yes", padx=10, pady=10)

wrapper2=LabelFrame(root, text="Basic Mineral Composition ")
wrapper2.pack(fill="both", expand="yes", padx=10, pady=5)

wrapper3=LabelFrame(root, text="Geographical Features")
wrapper3.pack(fill="both", expand="yes", padx=10, pady=5)

wrapper4=LabelFrame(root, text="Crop Suggestion Result")
wrapper4.pack(fill="both", expand="yes", padx=10, pady=5)

# Segment 1 : Preview Result

In [ ]:
#=============================Wrapper 1: Display Forecast Result=================================#

trv = ttk.Treeview(wrapper1, columns=(1,2,3,4,5,6,7,8), show="headings", height="15")
style=ttk.Style(trv)
style.configure('Treeview', rowheight=25)

trv.pack(side=LEFT)
trv.place(x=0, y=0)
trv.heading('#1', text="Nitrogen")
trv.heading('#2', text="Phosphorus")
trv.heading('#3', text="Pottasium")
trv.heading('#4', text="Temperature")
trv.heading('#5', text="Humidity")
trv.heading('#6', text="ph Level")
trv.heading('#7', text="Rainfall")
trv.heading('#8', text="Crop")

trv.column('#1',anchor=CENTER, width=111, minwidth=35)
trv.column('#2',anchor=CENTER, width=111, minwidth=35)
trv.column('#3',anchor=CENTER, width=111, minwidth=35)
trv.column('#4',anchor=CENTER, width=111, minwidth=35)
trv.column('#5',anchor=CENTER, width=111, minwidth=35)
trv.column('#6',anchor=CENTER, width=111, minwidth=35)
trv.column('#7',anchor=CENTER, width=111, minwidth=35)
trv.column('#8',anchor=CENTER, width=111, minwidth=35)


#create doubleclick function. so if doubleclick any rows, can get the data.
trv.bind('<Double 1>', getrow)

#vertical scrollbar
yscrollbar= ttk.Scrollbar(wrapper1, orient="vertical", command=trv.yview)
yscrollbar.pack(side=RIGHT, fill="y")

#horizontal scrollbar
#xscrollbar=ttk.Scrollbar(wrapper1, orient="horizontal", command=trv.xview)
#xscrollbar.pack(side=BOTTOM, fill="x")

trv.configure( yscrollcommand=yscrollbar.set)

query = ('SELECT Nitrogen,Phosphorus,Pottasium,temperature,humidity,ph,rainfall,Crop FROM Crop_recommendation')
cursor.execute(query)
rows= cursor.fetchall()
update(rows)

#======================================Notes==========================================#
#For manually input data to do simulation. 
#The data can be any data whether get from insider analytics or from governance data
#the input data only consist siginificance variable included in the final model
#Rainfall, number of workers, global radiation, pruning and pesticides usage.
#This simulation is based on annual data provided from 2010 until 2020. 
#Therefore, the simulation can simulate a future outcome annually, and this simulation cannot be used for monthly due to inaccuracy.



# Segment 2 & 3 : Label, Input and Button

In [ ]:
#=======================Wrapper 2: User Input for Simulation===========================#

###===========================Label and entry here===================================###

##==================Wrapper 2 =====================================##

lblFertilizer = Label(wrapper2, text="Mineral Composition", bd=10, width= 20, anchor='w')
lblFertilizer .grid(row=0, column=0)

lblNitro = Label(wrapper2, text="Nitrogen", bd=10, width= 20, anchor='w')
lblNitro.grid(row=1, column=0)
entNitro= Entry(wrapper2, textvariable=varNitro)
entNitro.grid(row=1, column=1)

lblPhos = Label(wrapper2, text="Phosphorus", bd=10, width= 20, anchor='w')
lblPhos.grid(row=1, column=2)
entPhos= Entry(wrapper2, textvariable=varPhos)
entPhos.grid(row=1, column=3)

lblPot = Label(wrapper2, text="Pottasium", bd=10, width= 20, anchor='w')
lblPot.grid(row=1, column=4)
entPot= Entry(wrapper2, textvariable=varPot)
entPot.grid(row=1, column=5)




In [ ]:

##==========================Wrapper 3 ======================#
lblTemp = Label(wrapper3, text="Temperature", bd=10, width= 20, anchor='w')
lblTemp.grid(row=0, column=0)
entTemp= Entry(wrapper3, textvariable=varTemp)
entTemp.grid(row=0, column=1)

lblHum = Label(wrapper3, text="Humidity", bd=10, width= 20, anchor='w')
lblHum.grid(row=0, column=2)
entHum= Entry(wrapper3, textvariable=varHum)
entHum.grid(row=0, column=3)

lblPH = Label(wrapper3, text="PH Level", bd=10, width= 20, anchor='w')
lblPH.grid(row=1, column=0)
entPH= Entry(wrapper3, textvariable=varPH)
entPH.grid(row=1, column=1)

lblRain = Label(wrapper3, text="Rainfall", bd=10, width= 20, anchor='w')
lblRain.grid(row=1, column=2)
entRain= Entry(wrapper3, textvariable=varRain)
entRain.grid(row=1, column=3)




In [ ]:

#button

btnCalc = Button(wrapper3, text="Start Model", width=12, height=2, activebackground='white', command=simulation)
btnCalc.grid(row=0, column=4, padx=10,pady=10)

btnClear = Button(wrapper3, text="Clear",  width=12, height=2, activebackground='white',command=clear)
btnClear.grid(row=1, column=4, padx=10,pady=10)

btnExport = Button(wrapper3, text="Export CSV", width=12, height=2, activebackground='white',  command=exportcsv)
btnExport.grid(row=0, column=5, padx=10,pady=10)

btnExit = Button(wrapper3, text="Exit", width=12, height=2, activebackground='white', command=exit_program)
btnExit.grid(row=1, column=5, padx=10,pady=10)


# Segment 4 : Result 

In [ ]:
lblResult = Label(wrapper4, text="Crop Yield Suggestion", bd=10, width= 20, anchor='w')
lblResult.grid(row=0, column=0)
entResult_RF= Entry(wrapper4, textvariable=varCrop, state="readonly")
entResult_RF.grid(row=0, column=1)


# Close Main Frame

In [ ]:
root.mainloop()